In [1]:
import pickle

import pandas as pd
import mlflow
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
os.environ["AWS_PROFILE"] = "ArunG"

RUN_ID = os.getenv('RUN_ID','1e2d274ec76643ca8f9ff1992d0c0850')
logged_model = f's3://mlflow-artifacts-gansi/1/{RUN_ID}/artifacts/model'
# logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 1e2d274ec76643ca8f9ff1992d0c0850

In [16]:
x_new = dict(x1=[1, 2, 3], x2=[4, 5, 6])

In [17]:
x_new

{'x1': [1, 2, 3], 'x2': [4, 5, 6]}

In [20]:
dicts

,PU_DO,trip_distance
0,43_151,1.01
1,166_239,2.53
2,41_42,1.12
3,168_75,1.99
4,75_75,0.45
...,...,...
73903,81_90,17.63
73904,35_213,18.36
73905,74_69,2.50
73906,168_215,14.48


In [11]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]


    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [15]:
type(model)

mlflow.pyfunc.PyFuncModel

In [19]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

dicts = pd.DataFrame(prepare_dictionaries(df))
y_pred = model.predict(dicts)

c:\Users\ArunGanesan\mlops-zoomcamp-garun\envs\mldeploy\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


ValueError: X has 2 features, but RandomForestRegressor is expecting 13221 features as input.